##### Copyright 2024 Google LLC.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Video understanding with Gemini 2.0

<table align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/google-gemini/cookbook/blob/main/gemini-2/video_understanding.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
</table>

Gemini has from the begining been a multimodal model, capable of analyzing all sorts of medias using its [long context window](https://developers.googleblog.com/en/new-features-for-the-gemini-api-and-google-ai-studio/).

[Gemini 2.0](https://ai.google.dev/gemini-api/docs/models/gemini-v2) bring video analysis to a whole new level as illustrated in [this video](https://www.youtube.com/watch?v=Mot-JEU26GQ):


In [ ]:
#@title Building with Gemini 2.0: Video understanding
%%html
<iframe width="560" height="315" src="https://www.youtube.com/embed/Mot-JEU26GQ?si=pcb7-_MZTSi_1Zkw" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture; web-share" referrerpolicy="strict-origin-when-cross-origin" allowfullscreen></iframe>

This notebook will show you how to easily use Gemini to perform the same kind of video analysis. Each of them has different prompts that you can select using the dropdown, also feel free to experiment with your own.

You can also check the [live demo](https://aistudio.google.com/starter-apps/video) and try it on your own videos on [AI Studio](https://aistudio.google.com/starter-apps/video).

## Setup

This section install the SDK, set it up using your [API key](../quickstarts/Authentication.ipynb), imports the relevant libs, downloads the sample videos and upload them to Gemini.

Expand the section if you are curious, but you can also just run it (it should take a couple of minutes since there are large files) and go straight to the examples.

### Install SDK

The new **[Google Gen AI SDK](https://ai.google.dev/gemini-api/docs/sdks)** provides programmatic access to Gemini 2.0 (and previous models) using both the [Google AI for Developers](https://ai.google.dev/gemini-api/docs) and [Vertex AI](https://cloud.google.com/vertex-ai/generative-ai/docs/overview) APIs. With a few exceptions, code that runs on one platform will run on both. This means that you can prototype an application using the Developer API and then migrate the application to Vertex AI without rewriting your code.

More details about this new SDK on the [documentation](https://ai.google.dev/gemini-api/docs/sdks) or in the [Getting started](../gemini-2/get_started.ipynb) notebook.

In [ ]:
!pip install -U -q google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.9/111.9 kB 3.3 MB/s eta 0:00:00


### Setup your API key

To run the following cell, your API key must be stored it in a Colab Secret named `GOOGLE_API_KEY`. If you don't already have an API key, or you're not sure how to create a Colab Secret, see [Authentication](../quickstarts/Authentication.ipynb) for an example.

In [ ]:
from google.colab import userdata

GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

### Initialize SDK client

With the new SDK you now only need to initialize a client with you API key (or OAuth if using [Vertex AI](https://link_to_vertex_AI)). The model is now set in each call.

In [ ]:
from google import genai
from google.genai import types

client = genai.Client(api_key=GOOGLE_API_KEY)

### Select the Gemini 2.0 model

Video understanding works best Gemini 2.0 Flash model. You can also select former models to compare their behavior but it is recommended to use the 2.0 one.

For more information about all Gemini models, check the [documentation](https://ai.google.dev/gemini-api/docs/models/gemini) for extended information on each of them.


In [ ]:
model_name = "gemini-2.0-flash-exp" # @param ["gemini-1.5-flash-001","gemini-1.5-pro-002","gemini-2.0-flash-exp"] {"allow-input":true}

### System instructions

 With the new SDK, the `system_instructions` and the `model` parameters must be passed in all `generate_content` calls, so let's save them to not have to type them all the time.

In [ ]:
system_instructions = """
    When given a video and a query, call the relevant function only once with the appropriate timecodes and text for the video
  """


These system instructions make sure the TODO: add explanation

### Get sample videos

In [ ]:
# Load sample images
!wget https://storage.googleapis.com/generativeai-downloads/videos/Pottery.mp4 -O Pottery.mp4 -q
!wget https://storage.googleapis.com/generativeai-downloads/videos/Jukin_Trailcam_Videounderstanding.mp4 -O Trailcam.mp4 -q
!wget https://storage.googleapis.com/generativeai-downloads/videos/post_its.mp4 -O Post_its.mp4 -q
!wget https://storage.googleapis.com/generativeai-downloads/videos/user_study.mp4 -O User_study.mp4 -q

### Upload the videos

Upload all the videos using the File API. You can find modre details about how to use it in the [Get Started](../gemini-2/get_started.ipynb#scrollTo=KdUjkIQP-G_i) notebook.

This can take a couple of minutes as the videos will need to be processed and tokenized.

In [ ]:
import time

def upload_video(video_file_name):
  video_file = client.files.upload(path=video_file_name)

  while video_file.state == "PROCESSING":
      print('Waiting for video to be processed.')
      time.sleep(10)
      video_file = client.files.get(name=video_file.name)

  if video_file.state == "FAILED":
    raise ValueError(video_file.state)
  print(f'Video processing complete: ' + video_file.uri)

  return video_file

pottery_video = upload_video('Pottery.mp4')
trailcam_video = upload_video('Trailcam.mp4')
post_its_video = upload_video('Post_its.mp4')
user_study_video = upload_video('User_study.mp4')

Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/hc3novqg1cb9
Waiting for video to be processed.
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/gtlhr7ubnxnp
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/eqh0uo7mjbt9
Waiting for video to be processed.
Video processing complete: https://generativelanguage.googleapis.com/v1beta/files/bwfach9o4ay1


### Imports

In [ ]:
import json
from PIL import Image
from IPython.display import display, Markdown, HTML

# Search within videos

First, try using the model to search within your videos and describe all the animal sightings in the trailcam video.

<video controls width="500"><source src="https://storage.googleapis.com/generativeai-downloads/videos/Jukin_Trailcam_Videounderstanding.mp4" type="video/mp4"></video>

In [ ]:
prompt = "For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object with the timecode of the caption in the video."  # @param ["For each scene in this video, generate captions that describe the scene along with any spoken text placed in quotation marks. Place each caption into an object with the timecode of the caption in the video.", "Organize all scenes from this video in a table, along with timecode, a short description, a list of objects visible in the scene (with representative emojis) and an estimation of the level of excitement on a scale of 1 to 10"] {"allow-input":true}

video = trailcam_video # @param ["trailcam_video", "pottery_video", "post_its_video", "user_study_video"] {"type":"raw","allow-input":true}

response = client.models.generate_content(
    model=model_name,
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=video.uri,
                    mime_type=video.mime_type),
                ]),
        prompt,
    ]
)

Markdown(response.text)

```json
[
  {
    "timecode": "00:00:00",
    "caption": "A close up of brown fur with visible plant life in the background."
  },
  {
    "timecode": "00:00:00",
    "caption": "A gray fox walks into the scene, followed by another fox. The background is a rocky area with some undergrowth. A high pitched sound effect can be heard."
  },
  {
    "timecode": "00:00:16",
     "caption": "A grayscale shot of a mountain lion in the forest, walking on some leaves."
  },
  {
     "timecode": "00:00:35",
     "caption": "A nighttime grayscale shot of two foxes in the forest. They appear to be digging."
  },
    {
     "timecode": "00:00:50",
      "caption": "An overexposed, blurry shot that is most of one solid color, followed by"
  },
  {
    "timecode": "00:00:51",
    "caption": "A nighttime shot of some foxes by rocks in a forest. A sound effect can be heard."
  },
  {
    "timecode": "00:01:04",
     "caption": "A nighttime shot of a mountain lion in the forest with visible rock formations."
  },
   {
    "timecode": "00:01:28",
      "caption": "A night time shot of a bobcat standing by a tree in the forest."
  },
  {
      "timecode": "00:01:51",
      "caption": "A medium shot of a black bear in the forest during the day. It turns to its left and moves past the camera."
  },
  {
    "timecode": "00:01:56",
    "caption": "A grayscale shot of a mountain lion moving quickly across the screen in a forest."
  },
    {
    "timecode": "00:02:05",
     "caption":"A close up of brown fur. The camera moves to reveal a black bear."
   },
 {
    "timecode": "00:02:22",
    "caption": "A night time grayscale shot of a fox looking at a lit up city in the background."
  },
    {
    "timecode": "00:02:35",
    "caption": "A nighttime grayscale shot of a bear walking near the edge of a cliff."
  },
 {
    "timecode": "00:02:51",
    "caption": "A nighttime shot of a mountain lion in the forest."
  },
{
    "timecode": "00:03:04",
    "caption":"A black bear is standing in the forest looking directly at the camera. The bear is moving. Then a lighter brown bear enters the frame from the right."
    },
    {
       "timecode": "00:03:22",
       "caption": "A lighter brown bear stands in the forest, looking to the left."
    },
    {
     "timecode": "00:04:21",
    "caption": "A nighttime shot of a bobcat with glowing eyes standing near a fallen tree limb."
  }
]
```

The prompt used is quite a generic one, but you can get even better results if you cutomize it to your needs (like asking specifically for foxes).

The [live demo on AI Studio](https://aistudio.google.com/starter-apps/video) shows how you can postprocess this output to jump directly to the the specific part of the video by clicking on the timecodes. If you are interested, you can check the [code of that demo on Github](https://github.com/google-gemini/starter-applets/tree/main/video).

# Extract and organize text

Gemini can also read what's in the video and extract it in an organized way. You can even use Gemini reasoning capabilities to generate new ideas for you.

<video controls width="400"><source src="https://storage.googleapis.com/generativeai-downloads/videos/post_its.mp4" type="video/mp4"></video>

In [ ]:
prompt = "Transcribe the sticky notes, organize them and put it in a table. Can you come up with a few more ideas?" # @param ["Transcribe the sticky notes, organize them and put it in a table. Can you come up with a few more ideas?", "Which of those names who fit an AI product that can resolve complex questions using its thinking abilities?"] {"allow-input":true}

video = post_its_video # @param ["trailcam_video", "pottery_video", "post_its_video", "user_study_video"] {"type":"raw","allow-input":true}

response = client.models.generate_content(
    model=model_name,
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=video.uri,
                    mime_type=video.mime_type),
                ]),
        prompt,
    ]
)

Markdown(response.text)

Certainly! Here's a table of the sticky notes, organized into categories based on their subject matter, along with a few ideas for additional considerations:

**Project Name Brainstorm Sticky Notes**

| Category            | Sticky Note Names    |
|--------------------|--------------------|
| **Constellations**  | Canis Major, Leo Minor, Lynx, Delphinus, Serpens, Centaurus, Lyra |
| **Celestial Objects**| Lunar Eclipse,  Stellar Nexus,  Comets Tail, Supernova Echo, Orion's Belt,   Celestial Drift|
| **Mythological**    |  Prometheus Rising, Chimera Dream, Perseus Shield, Zephyr, Odin, Aether, Athena's Eye, Athena, Medusa, Hera, Cerberus, |
| **Scientific Concepts** | Convergence,  Astral Forge,  Draco,  Euler's Path, Equilibrium,  Chaos Theory, Golden Ratio, Infinity Loop, Taylor Series, Stokes Theorem,  Bayes Theorem, Riemann's Hypothesis, Fractal, Symmetry, Vector |
| **Other** |   Galactic Core, Titan, Echo, Phoenix, Pandora's Box, Sagitta|


**Ideas for What We May Have Missed**

1. **Themes & Subcategories**: Is there a desired theme or subcategory (e.g., "geometric", "mythological", "technical", etc.)? If so, some additional terms or more targeted brainstorming in that area might be beneficial.
    * **Greek Mythology**: Expand on Greek/Roman myths. Consider names like: *Icarus's Fall*, *Pegasus*, *Hades' Gate*, *Poseidon's Trident*.
    * **Astronomy**: More technical names from astronomy like *Solar Flare*, *Accretion Disk*, *Gravitational Lens*, *Quasar Jet*, *Event Horizon*, etc.
2. **Style**:
    * **Simplicity vs. Complexity**:  Do you want a name that is straightforward or something more complex and evocative?  Consider a combination of scientific and mythological names.
    * **Acronyms/Initialisms**: Some project names are short acronyms (e.g., ALICE, ATLAS). Perhaps brainstorm some of those if you need a shorter title.
3. **Audience**: Who is the audience for this project? The name could be more or less technical, depending on whether it's an expert audience or a general public one.
4. **Uniqueness**: It's often valuable to research if any of these project names are already in use to avoid confusion.
5. **Team Name/Internal vs. External Name**: Is this a team name or is this a name for a product that the public will see?

Let me know if you’d like me to explore any of these ideas or categories in more detail!

# Structure information

Gemini 2.0 is not only able to read text but also to reason and structure about real world objects. Like in this video about a display of ceramics with handwritten prices and notes.

<video controls width="500"><source src="https://storage.googleapis.com/generativeai-downloads/videos/Pottery.mp4" type="video/mp4"></video>

In [ ]:
prompt = "Give me a table of my items and notes" # @param ["Give me a table of my items and notes", "Help me come up with a selling pitch for my potteries"] {"allow-input":true}

video = pottery_video # @param ["trailcam_video", "pottery_video", "post_its_video", "user_study_video"] {"type":"raw","allow-input":true}

response = client.models.generate_content(
    model=model_name,
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=video.uri,
                    mime_type=video.mime_type),
                ]),
        prompt,
    ],
    config = types.GenerateContentConfig(
        system_instruction="Don't forget to escape the dollar signs",
    )
)

Markdown(response.text)

As you can see, Gemini is able to grasp to with item corresponds each note, including the last one.

# Analyze screen recordings for key moments

You can also use the model to analyze screen recordings. Let's say you're doing user studies on how people use your product, so you end up with lots of screen recordings, like this one, that you have to manually comb through.
With just one prompt, the model can describe all the actions in your video.

<video controls width="400"><source src="https://storage.googleapis.com/generativeai-downloads/videos/user_study.mp4" type="video/mp4"></video>

In [ ]:
prompt = "Generate a paragraph that summarizes this video. Keep it to 3 to 5 sentences with corresponding timecodes." # @param ["Generate a paragraph that summarizes this video. Keep it to 3 to 5 sentences with corresponding timecodes.", "Choose 5 key shots from this video and put them in a table with the timecode, text description of 10 words or less, and a list of objects visible in the scene (with representative emojis).", "Generate bullet points for the video. Place each bullet point into an object with the timecode of the bullet point in the video."] {"allow-input":true}

video = user_study_video # @param ["trailcam_video", "pottery_video", "post_its_video", "user_study_video"] {"type":"raw","allow-input":true}

response = client.models.generate_content(
    model=model_name,
    contents=[
        types.Content(
            role="user",
            parts=[
                types.Part.from_uri(
                    file_uri=video.uri,
                    mime_type=video.mime_type),
                ]),
        prompt,
    ],
)

Markdown(response.text)

Once again, you can check the  [live demo on AI Studio](https://aistudio.google.com/starter-apps/video) shows an example on how to postprocess this output. Check the [code of that demo](https://github.com/google-gemini/starter-applets/tree/main/video) for more details.

# Next Steps

Try with you own videos using the [AI Studio's live demo](https://aistudio.google.com/starter-apps/video) or play with the examples from this notebook (in case you haven't seen, there are other prompts you can try in the dropdowns).

For more examples of the Gemini 2.0 capabilities, check the [Gemini 2.0 folder of the cookbook](https://github.com/google-gemini/cookbook/blob/main/gemini-2/). You'll learn how to use the [Live API](live_api_starter.ipynb), juggle with [multiple tools](./plotting_and_mapping.ipynb) or use Gemini 2.0 [spatial understanding](./spatial_understanding.ipynb) abilities.

The [examples](../examples) folder from the cookbook is also full of nice code samples illustrating creative ways to use Gemini multimodal capabilities and long-context.